# Medical transcript classification [sklearn]
* Multiclass classification of medical transcript.
* Reference notebook: <https://www.kaggle.com/code/leekahwin/text-classification-using-n-gram-0-8-f1/notebook>
* Dataset: <https://www.kaggle.com/code/leekahwin/text-classification-using-n-gram-0-8-f1/input>

By running this notebook, you’ll create a whole test suite in a few lines of code. The model used here is a Random Forest classification model with the medical transcript dataset. Feel free to use your own model (tabular, text, or LLM).

You’ll learn how to:
* Detect vulnerabilities by scanning the model
* Generate a test suite with domain-specific tests
* Customize your test suite by loading a test from the Giskard catalog
* Upload your model to the Giskard server to:
    * Compare models to decide which one to promote
    * Debug your tests to diagnose issues
    * Share your results and collect business feedback from your team

## Install Giskard

In [ ]:
pip install "giskard>=2.0.0b" -U

## Install necessary dependencies

In [ ]:
!pip install nltk

## Import libraries

In [1]:
import os
import string
from pathlib import Path
from typing import Iterable
from urllib.request import urlretrieve

import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

import giskard
from giskard import Dataset, Model, GiskardClient, testing

## Download NLTK stopwords corpus

In [ ]:
# Download list of english stopwords.
nltk.download('stopwords')

## Define constants

In [2]:
# Constants.
LABELS_LIST = [
    'Neurosurgery',
    'ENT - Otolaryngology',
    'Discharge Summary',
    'General Medicine',
    'Gastroenterology',
    'Neurology',
    'SOAP / Chart / Progress Notes',
    'Obstetrics / Gynecology',
    'Urology'
]

TEXT_COLUMN_NAME = "transcription"
TARGET_COLUMN_NAME = "medical_specialty"

RANDOM_SEED = 8888

# Data.
DATA_URL = os.path.join("ftp://sys.giskard.ai", "pub", "unit_test_resources",
                        "medical_transcript_classification_dataset", "mtsamples.csv")
DATA_PATH = Path.home() / ".giskard" / "medical_transcript_classification_dataset" / "mtsamples.csv"

## Dataset preparation

### Load data

In [3]:
def fetch_from_ftp(url: str, file: Path) -> None:
    """Helper to fetch data from the FTP server."""
    if not file.parent.exists():
        file.parent.mkdir(parents=True, exist_ok=True)

    if not file.exists():
        print(f"Downloading data from {url}")
        urlretrieve(url, file)

    print(f"Data was loaded!")


def load_data() -> pd.DataFrame:
    """Load and initially preprocess data."""
    fetch_from_ftp(DATA_URL, DATA_PATH)

    df = pd.read_csv(DATA_PATH)

    # Drop useless columns.
    df = df.drop(columns=['Unnamed: 0', "description", "sample_name", "keywords"])

    # Trim text.
    df = df.apply(lambda x: x.str.strip())

    # Filter samples by label.
    df = df[df[TARGET_COLUMN_NAME].isin(LABELS_LIST)]

    # Drop rows with no transcript.
    df = df[df[TEXT_COLUMN_NAME].notna()]

    return df

In [4]:
transcript_df = load_data()

Data was loaded!


### Train-test split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(transcript_df[[TEXT_COLUMN_NAME]],
                                                    transcript_df[TARGET_COLUMN_NAME],
                                                    random_state=RANDOM_SEED)

### Wrap dataset with Giskard

In [6]:
raw_data = pd.concat([X_test, y_test], axis=1)
wrapped_data = Dataset(raw_data,
                       name="medical_transcript_dataset",
                       target=TARGET_COLUMN_NAME,
                       column_types={TEXT_COLUMN_NAME: "text"})

## Model training

### Define preprocessing steps

In [7]:
stemmer = SnowballStemmer("english")
stop_words = stopwords.words("english")


def preprocess_text(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess text."""
    # Lower.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: x.lower())

    # Remove punctuation.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

    # Tokenize.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: x.split())

    # Stem.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: [stemmer.stem(word) for word in x])

    # Remove stop-words.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(
        lambda x: ' '.join([word for word in x if word not in stop_words]))

    return df


def adapt_vectorizer_input(df: pd.DataFrame) -> Iterable:
    """Adapt input for the vectorizers.

    The problem is that vectorizers accept iterable, not DataFrame, but Series. Thus, we need to ravel dataframe with text have input single dimension.
    Issue reference: https://stackoverflow.com/questions/50665240/valueerror-found-input-variables-with-inconsistent-numbers-of-samples-1-3185"""

    df = df.iloc[:, 0]
    return df


text_preprocessor = FunctionTransformer(preprocess_text)
vectorizer_input_adapter = FunctionTransformer(adapt_vectorizer_input)

### Build estimator

In [8]:
pipeline = Pipeline(steps=[
    ("text_preprocessor", text_preprocessor),
    ("vectorizer_input_adapter", vectorizer_input_adapter),
    ("vectorizer", CountVectorizer(ngram_range=(1, 1))),
    ("estimator", RandomForestClassifier(random_state=RANDOM_SEED))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

                               precision    recall  f1-score   support

            Discharge Summary       0.43      0.38      0.41        34
         ENT - Otolaryngology       1.00      0.59      0.74        27
             Gastroenterology       0.57      0.81      0.67        43
             General Medicine       0.46      0.61      0.52        69
                    Neurology       0.74      0.66      0.70        53
                 Neurosurgery       0.73      0.79      0.76        24
      Obstetrics / Gynecology       0.85      0.58      0.69        50
SOAP / Chart / Progress Notes       0.40      0.36      0.38        33
                      Urology       0.74      0.68      0.71        38

                     accuracy                           0.61       371
                    macro avg       0.66      0.61      0.62       371
                 weighted avg       0.64      0.61      0.62       371



### Wrap model with Giskard

In [9]:
wrapped_model = Model(pipeline.predict_proba,
                      model_type="classification",
                      name="medical_transcript_classification",
                      feature_names=[TEXT_COLUMN_NAME],
                      classification_labels=pipeline.classes_)

# Validate wrapped model and data.
print(classification_report(y_test, pipeline.classes_[wrapped_model.predict(wrapped_data).raw_prediction]))

                               precision    recall  f1-score   support

            Discharge Summary       0.43      0.38      0.41        34
         ENT - Otolaryngology       1.00      0.59      0.74        27
             Gastroenterology       0.57      0.81      0.67        43
             General Medicine       0.46      0.61      0.52        69
                    Neurology       0.74      0.66      0.70        53
                 Neurosurgery       0.73      0.79      0.76        24
      Obstetrics / Gynecology       0.85      0.58      0.69        50
SOAP / Chart / Progress Notes       0.40      0.36      0.38        33
                      Urology       0.74      0.68      0.71        38

                     accuracy                           0.61       371
                    macro avg       0.66      0.61      0.62       371
                 weighted avg       0.64      0.61      0.62       371



## Scan your model to find vulnerabilities
With the Giskard scan feature, you can detect vulnerabilities in your model, including performance biases, unrobustness, data leakage, stochasticity, underconfidence, ethical issues, and more. For detailed information about the scan feature, please refer to our scan documentation.

In [10]:
results = giskard.scan(wrapped_model, wrapped_data)

Your model is successfully validated.
Running scan…
2023-06-02 17:57:04,714 pid:10102 MainThread giskard.scanner.logger INFO     Running detectors: ['PerformanceBiasDetector', 'TextPerturbationDetector', 'EthicalBiasDetector', 'DataLeakageDetector', 'StochasticityDetector', 'OverconfidenceDetector', 'UnderconfidenceDetector']
Running detector PerformanceBiasDetector…2023-06-02 17:57:04,715 pid:10102 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Running
2023-06-02 17:57:04,715 pid:10102 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Calculating loss
2023-06-02 17:57:05,038 pid:10102 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Loss calculated (took 0:00:00.321848)
2023-06-02 17:57:05,038 pid:10102 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Finding data slices
2023-06-02 17:58:00,022 pid:10102 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: 321 slices found (took 0:00:54.982054)

In [11]:
display(results)

<iframe id="scan-4947914384" srcdoc="
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collapse:collapse;border-color:inherit;text-indent:0}button,input,optgroup,select,textarea{color:inherit;font-family:inherit;font-size:100%

## Generate a test suite from the Scan
The objects produced by the scan can be used as fixtures to generate a test suite that integrate domain-specific issues. To create custom tests, refer to the Test your ML Model page.

In [12]:
test_suite = results.generate_test_suite("My first test suite")
test_suite.run()

Executed 'F1 Score on data slice “`transcription` contains "xyz"”' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x12245e380>, 'dataset': <giskard.datasets.base.Dataset object at 0x122397c70>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x126210cd0>, 'threshold': 0.581266846361186}: 
               Test failed
               Metric: 0.32
               
               
Executed 'F1 Score on data slice “`transcription` contains "subjective"”' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x12245e380>, 'dataset': <giskard.datasets.base.Dataset object at 0x122397c70>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x1261111b0>, 'threshold': 0.581266846361186}: 
               Test failed
               Metric: 0.37
               
               
Executed 'F1 Score on data slice “`transcription` contains "admission"”' with arguments {'model': <giskard.

(False,
 [('F1 Score on data slice “`transcription` contains "xyz"”', 
                  Test failed
                  Metric: 0.32
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "subjective"”',
   
                  Test failed
                  Metric: 0.37
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "admission"”',
   
                  Test failed
                  Metric: 0.38
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "daily"”',
   
                  Test failed
                  Metric: 0.38
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "coronary"”',
   
                  Test failed
                  Metric: 0.39
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "followup"”',
   
                  Test failed
                  Metric: 0.4
                  
                  ),
  ('Precision on data slice “`transcription` contains "count"”',
   
                  Test failed
                  Metric: 0.4
                  
                  ),
  ('Precision on data slice “`transcription` contains "lung"”',
   
                  Test failed
                  Metric: 0.4
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "function"”',
   
                  Test failed
                  Metric: 0.41
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "abc"”',
   
                  Test failed
                  Metric: 0.42
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "improved"”',
   
                  Test failed
                  Metric: 0.42
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "discharge"”',
   
                  Test failed
                  Metric: 0.42
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "aspirin"”',
   
                  Test failed
                  Metric: 0.43
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "greater"”',
   
                  Test failed
                  Metric: 0.43
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "continue"”',
   
                  Test failed
                  Metric: 0.44
                  
                  ),
  ('Invariance to “Add typos”',
   
                  Test succeed
                  Metric: 0.96
                   - [TestMessageLevel.INFO] 369 rows were perturbed
                  )])

## Customize your suite by loading objects from the Giskard catalog

The Giskard open source catalog will enable to load:
* Tests such as metamorphic, performance, prediction & data drift, statistical tests, etc
* Slicing functions such as detectors of toxicity, hate, emotion, etc
* Transformation functions such as generators of typos, paraphrase, style tune, etc

For demo purposes, we will load a simple unit test (test_f1) that checks if the test F1 score is above the given threshold. For more examples of tests and functions, refer to the Giskard catalog.

In [14]:
test_suite.add_test(testing.test_f1(model=wrapped_model, dataset=wrapped_data, threshold=0.7)).run()

Executed 'F1 Score on data slice “`transcription` contains "xyz"”' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x12245e380>, 'dataset': <giskard.datasets.base.Dataset object at 0x122397c70>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x126210cd0>, 'threshold': 0.581266846361186}: 
               Test failed
               Metric: 0.32
               
               
Executed 'F1 Score on data slice “`transcription` contains "subjective"”' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x12245e380>, 'dataset': <giskard.datasets.base.Dataset object at 0x122397c70>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x1261111b0>, 'threshold': 0.581266846361186}: 
               Test failed
               Metric: 0.37
               
               
Executed 'F1 Score on data slice “`transcription` contains "admission"”' with arguments {'model': <giskard.

(False,
 [('F1 Score on data slice “`transcription` contains "xyz"”', 
                  Test failed
                  Metric: 0.32
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "subjective"”',
   
                  Test failed
                  Metric: 0.37
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "admission"”',
   
                  Test failed
                  Metric: 0.38
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "daily"”',
   
                  Test failed
                  Metric: 0.38
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "coronary"”',
   
                  Test failed
                  Metric: 0.39
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "followup"”',
   
                  Test failed
                  Metric: 0.4
                  
                  ),
  ('Precision on data slice “`transcription` contains "count"”',
   
                  Test failed
                  Metric: 0.4
                  
                  ),
  ('Precision on data slice “`transcription` contains "lung"”',
   
                  Test failed
                  Metric: 0.4
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "function"”',
   
                  Test failed
                  Metric: 0.41
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "abc"”',
   
                  Test failed
                  Metric: 0.42
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "improved"”',
   
                  Test failed
                  Metric: 0.42
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "discharge"”',
   
                  Test failed
                  Metric: 0.42
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "aspirin"”',
   
                  Test failed
                  Metric: 0.43
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "greater"”',
   
                  Test failed
                  Metric: 0.43
                  
                  ),
  ('F1 Score on data slice “`transcription` contains "continue"”',
   
                  Test failed
                  Metric: 0.44
                  
                  ),
  ('Invariance to “Add typos”',
   
                  Test failed
                  Metric: 0.94
                   - [TestMessageLevel.INFO] 371 rows were perturbed
                  ),
  ('F1',
   
                  Test failed
                  Metric: 0.61
                  
                  ),
  ('F1',
   
                  Test failed
                  Metric: 0.61
                  
                  )])

## Upload your suite to the Giskard server

Upload your suite to the Giskard server to:
* Compare models to decide which model to promote
* Debug your tests to diagnose the issues
* Create more domain-specific tests that are integrating business feedback
* Share your results

In [ ]:
# Uploading the test suite will automatically save the model, dataset, tests, slicing & transformation functions inside the Giskard UI server
# Create a Giskard client after having install the Giskard server (see documentation)
token = "API_TOKEN"  # Find it in Settings in the Giskard server

client = GiskardClient(
    url="http://localhost:19000",  # URL of your Giskard instance
    token=token
)

my_project = client.create_project("my_project", "PROJECT_NAME", "DESCRIPTION")

# Upload to the current project ✉️
test_suite.upload(client, "my_project")

<div class="alert alert-info">
Connecting Google Colab with the Giskard server

If you are using Google Colab and you want to install the Giskard server **locally**, you can run the Giskard server by executing this line in the terminal of your **local** machine (see the [documentation]https://docs.giskard.ai/en/latest/guides/installation_app/index.html)):

> giskard server start

Once the Giskard server is running, from the same terminal on your **local** machine, you can run:

> giskard server expose

This will provide you with the code snippets that you can copy and paste into your Colab notebook to establish a connection with your locally installed Giskard server
</div>